In [1]:
from datasets import load_dataset
from transformers import BertTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EncoderDecoderModel, BertLMHeadModel, BertConfig
import evaluate
from custom_transformer.src.transformers.models.bert.modeling_bert import BertModel

c:\Users\Alienware\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_PATH = r"C:\Users\Alienware\Documents\Project\RoBERTA-ViAttention\merged_vIErr2_and_ViLexNorm.csv"
dataset = load_dataset("csv", data_files=DATASET_PATH)
MODEL_NAME = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [3]:
def process_function(examples):
    model_inputs = tokenizer(examples["input"], truncation=True, padding="max_length", max_length=50)
    labels = tokenizer(examples["output"], truncation=True, padding="max_length", max_length=50)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(process_function, batched=True)

In [ ]:
tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2) 
tokenized_dataset 
train_size = int(0.05 * len(tokenized_dataset["train"]))
eval_size = int(0.05 * len(tokenized_dataset["test"]))

small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(train_size))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(eval_size))

In [5]:
encoder = BertModel.from_pretrained(MODEL_NAME)
decoder_config = BertConfig.from_pretrained(MODEL_NAME)
decoder_config.is_decoder = True 
decoder_config.add_cross_attention = True  

decoder = BertLMHeadModel.from_pretrained("bert-base-uncased", config=decoder_config)
print("Encoder and decoder loaded")
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)

Some weights of BertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['encoder.layer.0.attention.self.W', 'encoder.layer.1.attention.self.W', 'encoder.layer.10.attention.self.W', 'encoder.layer.11.attention.self.W', 'encoder.layer.2.attention.self.W', 'encoder.layer.3.attention.self.W', 'encoder.layer.4.attention.self.W', 'encoder.layer.5.attention.self.W', 'encoder.layer.6.attention.self.W', 'encoder.layer.7.attention.self.W', 'encoder.layer.8.attention.self.W', 'encoder.layer.9.attention.self.W']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.cr

Encoder and decoder loaded


In [8]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\Alienware\AppData\Local\Temp\ipykernel_22332\1081152546.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
